In [1]:
import pandas as pd
import numpy as np

In [2]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [17]:
# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data Reading

In [4]:
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [30]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(_read_csv(filename).values)

    return np.transpose(signals_data, (1, 2, 0))

In [31]:
def load_y(subset):
    filename = f'UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [15]:
def load_data():
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

Using TensorFlow backend.


In [10]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [11]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [32]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [33]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [34]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [36]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 17s 2ms/step - loss: 1.3267 - accuracy: 0.4525 - val_loss: 1.1931 - val_accuracy: 0.4394
Epoch 2/30
7352/7352 [==============================] - 16s 2ms/step - loss: 1.0063 - accuracy: 0.5554 - val_loss: 0.9284 - val_accuracy: 0.5728
Epoch 3/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.7935 - accuracy: 0.6432 - val_loss: 0.8548 - val_accuracy: 0.5891
Epoch 4/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.6878 - accuracy: 0.6647 - val_loss: 0.6727 - val_accuracy: 0.6400
Epoch 5/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.6302 - accuracy: 0.7142 - val_loss: 0.6955 - val_accuracy: 0.7207
Epoch 6/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.5577 - accuracy: 0.7896 - val_loss: 0.5817 - val_accuracy: 0.7906
Epoch 7/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.455

In [37]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 519        0        18        0                   0   
SITTING                  0      379       110        0                   2   
STANDING                 0       62       464        6                   0   
WALKING                  0        0         0      459                  37   
WALKING_DOWNSTAIRS       0        0         0        3                 414   
WALKING_UPSTAIRS         0        0         0       34                  28   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 409  


In [38]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 234us/step


In [39]:
score

[0.5955118426720399, 0.8971835970878601]

Due to less data sample LSTM model is less performed

In [42]:
model.save("HR LSTM.h5")